# LSTM for Trump text generation

![alt text](https://miro.medium.com/v2/resize:fit:984/1*Mb_L_slY9rjMr8-IADHvwg.png)

Tensorflow will be used for the integration of the LSTM model.

## Dataset:

In [1]:
import os

path = 'Trump Rally Speeches/'
files = os.listdir(path)
files = [path + file for file in files]
 
dates = []
locations = []
years = []
days = []
months = []
speeches_text = []
 
month_ab = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec']

for file in files:
    for month in month_ab:
        if month in file:
            locations.append(file[file.find('/')+1:file.find(month)])
            break
    for i, mont in enumerate(month_ab):
        if month in file:
            date = file[file.find(month):file.find('.txt')]
            dates.append(date)
            months.append(date[:3])
            days.append(str(date[3]))
            years.append(date[-4:])
            break   
        
for file in files:
    with open(file, 'r') as f:
        speeches_text.append(f.read())     
        
import pandas as pd
 
df = pd.DataFrame({'Speech':files, 'Date':dates, 'Location':locations, 'Year':years, 'Month':months, 'Day':days, 'Speech_Text':speeches_text})

In [2]:
from preprocessing import preprocessing_pipline

preprocessing = preprocessing_pipline(df['Speech_Text'])
df['Speech_Text_prepro'] = preprocessing.preprocess_light()
print 

thank thank thank vice president pence hes good guy weve done great job together merry christmas mic
Thank you. Thank you. Thank you to Vice President Pence. He's a good guy. We've done a great job tog
thank thank thank vice president pence hes good guy weve done great job together merry christmas mic


In [13]:
from sklearn.model_selection import train_test_split

text_corpus = [word for speech in df['Speech_Text'].str.split() for word in speech]
train_corpus, test_corpus = train_test_split(text_corpus, test_size=0.2, random_state=42)
print("text corpus type",type(text_corpus))
print("text corpus type",type(text_corpus[0]))


# Preprocess text 
text_corpus_prepro = [word for speech in df['Speech_Text_prepro'].str.split() for word in speech]
train_corpus_prepro, test_corpus_prepro = train_test_split(text_corpus_prepro, test_size=0.2, random_state=42)

text corpus type <class 'list'>
text corpus type <class 'str'>


In [74]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np


def preprocess_for_LSTM(text_corpus, max_sequence_length=10, test_size=0.2):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_corpus)

    input = []
    output = []

    lenght_of_sequences = 10

    for i in range(lenght_of_sequences, len(train_corpus)):
        input.append(train_corpus[i-lenght_of_sequences:i])
        output.append(train_corpus[i])

    input = tokenizer.texts_to_sequences(input)
    output = tokenizer.texts_to_sequences(output)

    # pad 
    input = pad_sequences(input, maxlen=lenght_of_sequences, padding='pre')
    output = pad_sequences(output, maxlen=1, padding='pre')
    # Convert your lists of input sequences into numpy arrays
    
    input = np.array(input)
    output = np.array(output)

    # Split your data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, tokenizer

 

In [76]:
X_train, X_test, y_train, y_test, tokenizer = preprocess_for_LSTM(text_corpus_prepro, max_sequence_length=10, test_size=0.2)

## The model

In [77]:
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dense

class my_model_LSTM:
    def __init__(self, n, num_unique_words, max_sequence_length):
        self.n = n
        self.num_unique_words = num_unique_words
        self.max_sequence_length = max_sequence_length
        self.model = self.build_model()
        
    def build_model(self):
        model = tf.keras.Sequential()
        model.add(Embedding(self.num_unique_words, self.n))
        model.add(LSTM(units=self.n))
        model.add(Dense(units=self.num_unique_words, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
     
    def train(self, X_train, y_train, epochs=10):
        self.model.fit(X_train, y_train, epochs=epochs)
         
    def predict(self, test_corpus):
        return self.model.predict(test_corpus)
     
    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test)
     
    def save(self, path):
        self.model.save(path)
         
    def load(self, path):
        self.model = tf.keras.models.load_model(path)
         
    def generate_text(self, seed_text, max_length=100):
        output_text = seed_text.split()
        prefix = output_text[-(self.n - 1):]

        for _ in range(max_length):
            input_eval = [tokenizer.word_index[word] for word in prefix]
            input_eval = tf.expand_dims(input_eval, 0)

            predictions = self.model(input_eval)
            predictions = tf.squeeze(predictions, 0)

            if len(predictions.shape) == 1:
                predictions = tf.expand_dims(predictions, 0)

            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
            next_word = tokenizer.index_word[predicted_id]

            output_text.append(next_word)
            prefix = prefix[1:] + [next_word]

        return ' '.join(output_text)



In [78]:
num_unique_words = len(tokenizer.word_index) + 1
max_sequence_length = 15
model_LSTM = my_model_LSTM(256, num_unique_words, max_sequence_length)
model_LSTM.train(X_train, y_train, epochs=3)

Epoch 1/3
7410/7410 [==============================] - 475s 64ms/step - loss: 4.2155 - accuracy: 0.5302
Epoch 2/3
7410/7410 [==============================] - 473s 64ms/step - loss: 4.1263 - accuracy: 0.5305
Epoch 3/3
7410/7410 [==============================] - 504s 68ms/step - loss: 4.1029 - accuracy: 0.5305


In [80]:
model_LSTM.save('model_LSTM.h5')

/workspaces/Trump_TextGeneration_NLP_Project/myenv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [79]:
predictions = model_LSTM.evaluate(X_test, y_test)
print("LSTM Model Accuracy:", predictions[1])
print("LSTM Model Loss:", predictions[0])

1853/1853 [==============================] - 27s 15ms/step - loss: 4.2061 - accuracy: 0.5303
LSTM Model Accuracy: 0.5302910208702087
LSTM Model Loss: 4.206118106842041


In [87]:
model_LSTM = my_model_LSTM(256, num_unique_words, max_sequence_length)
model_LSTM.load('model_LSTM.h5')
seed_text = 'france forcing'
generated_text = model_LSTM.generate_text(seed_text, max_length=10)
print(generated_text)

france forcing elko harrington welcoming achievement wisconsin dishonest la bystanders drip studios


## Model